# SQL-Python Integration

This notebook demonstrates the basic approach for using SQL in Python. It uses `sqlite3` for the reasons described in the accompanying slides.

You can find more details at the [Official SQLite Documentation](https://docs.python.org/3/library/sqlite3.html).

Run the following cell first to get the required data files. Though you may not know the methods used, you should, at this point, be able to follow the code and get a sense of what it does.

In [ ]:
from pathlib import Path
from urllib.request import urlretrieve

def download_if_missing(url, filename, data_dir = Path('data')):
    """
    Download a file from URL if not present in local data directory.
    
    Args:
        url: Source URL to download from
        filename: Name to save file as locally
        data_dir: Path to data directory (default: Path('data'))
    
    Returns:
        Path: Path object pointing to the local file
    """

    # make sure a data subdirectory exists
    data_dir.mkdir(exist_ok=True)

    # build a full Path objecct
    filepath = data_dir / filename

    if not filepath.exists():
        # there is no file in the data directory called filename
        print(f"Downloading {filename} to {data_dir}/...")
        try:
            urlretrieve(url, filepath)
            print("Download complete!")
        except Exception as e:
            print(f"Error downloading {filename}: {e}")
    else:
        print(f"{filepath} exists, skipping download.")

    return filepath

# define constants
DATA_DIR = Path('data')
BASE_URL = 'https://raw.githubusercontent.com/olearydj/INSY3010-Fall24/main/notebooks/data/'
FLIGHT_DB_FILE = 'flight.db'
TUTORIAL_DB_FILE = 'tutorial.db'

# download flight database if missing
flight_db = download_if_missing(BASE_URL + FLIGHT_DB_FILE, FLIGHT_DB_FILE, DATA_DIR)

# Delete tutorial database if exists
tutorial_db = DATA_DIR / TUTORIAL_DB_FILE
tutorial_db.unlink(missing_ok=True)

### Flight Database

#### Basic Query Operations

In [ ]:
# sqlite3 is included with python
import sqlite3

In [ ]:
# create a connection to the database
con = sqlite3.connect(flight_db)

# create a cursor object for the connection
cur = con.cursor()

# execute SQL commands with execute method
SQL = "SELECT * FROM flight LIMIT 10"
result = cur.execute(SQL)

# result is a Cursor object type
type(result)

In [ ]:
# can loop through rows...
for row in result:
    print(row)

In [ ]:
# or get all the results as a list of tuples using fetchall
data = cur.fetchall()
type(data)

In [ ]:
print(data[:2])

`data` is an empty list - What happened!?

Cursor objects are iterators that allow you to process query results sequentially. They provide an *interface* for fetching rows on demand rather than loading all results into memory at once. This makes them memory-efficient when dealing with large result sets, as you can process one row at a time rather than loading the entire result set at once.

In this case, the earlier loop exhausted the contents of the Cursor object. You can think of this as reaching the end of a linear physical media object like a record ('vinyl') or cassette tape. There is no  way to "rewind" in SQLite, except to run the query again:

In [ ]:
# re-run the query and fetch all the results
result = cur.execute(SQL)
data = cur.fetchall()

# the result is a list of tuples
print(type(data), type(data[0]))


In [ ]:
# check the first two lines
print(data[:2])

After storing the results of `fetchall` in a variable, you can use them multiple times.

In [ ]:
print(data[-1])

In [ ]:
len(data)

Note that the difference here is not the method used - in both cases we used `cur.fetchall()` - but the order of operations. In the first attempt we executed the query, printed the rows, then performed the fetch operation. The second time, we used `fetchall` immediately after executing the query.

In the first case we exausted the Cursor object before fetching the results, resulting in an empty list. The second time the list was created before "playing" any of the database records (rows). This is further demonstrated by the following example.

In [ ]:
# re-run the query to reset the Cursor object
result = cur.execute(SQL)

# print first 5 records using fetchone
for row in range(5):
    print(result.fetchone())

In [ ]:
# only 5 records remain!
data = cur.fetchall()
len(data)

Be careful about when and how you fetch query results. In a live database, each time you execute the query to reset the Cursor the underlying data, and thus results, may change!

#### Multi-Line Queries

Python allows multi-line strings, which is useful for longer queries.  Just wrap the lines in triple quotes, either single or double.

In [ ]:
SQL = '''
SELECT FlightNumber, COUNT(*) AS Count
FROM booking
GROUP BY FlightNumber
'''

print(SQL)

In [ ]:
# re-run the query and fetch all the results
result = cur.execute(SQL)
data = cur.fetchall()

for record in data:
    print(record)

#### Attribute Names

To get the names of the table attributes (columns), use the `description` method. It returns a list of tuples where the first element in each tuple is the column name:

In [ ]:
for col in cur.description:
    col_name = col[0]
    print(col_name)

#### Close the Connection

Always close the database connection!

In [ ]:
con.close()

### Full CRUD

A short demonstration of how to create and add to tables.

Create a connection to a new or existing database using `sqlite3.connect()` with the path.

In [ ]:
# db file will be created if new
con = sqlite3.connect(tutorial_db)
print(con) # connection object

Create a "cursor" object for the connection using `.cursor()`. Wikipedia defines a database cursor as...

> a mechanism that enables traversal over the records in a database

This is similar to an interator for the database, and will be used as such.

In [ ]:
# create a "cursor" to iterate thru the database
cur = con.cursor()
print(cur) # cursor object

### Create a Table and Populate it with Data
We have focused on the **R** in **CRUD**, *reading* databases with `SELECT` statements. Here we *create* and *update* a database of movie data using `CREATE` and `INSERT` statements, followed by a `con.commit()` to save the changes.

Queries are run using the `cursor.execute()` method, which takes a string of the SQL command as an argument.

Note the use of parentheses with adjacent strings for multi-line string definition.

In [ ]:
# create a table of movie data
cur.execute("CREATE TABLE movie(title, year, score)")

# add some data...
# first, create a string holding the SQL statement to execute

SQL = '''
INSERT INTO movie VALUES
('Monty Python and the Holy Grail', 1975, 8.2),
('And Now for Something Completely Different', 1972, 7.5)
'''

# execute the SQL statement
cur.execute(SQL)

# commit the change to the database
con.commit()

### Test and Add More Data

The same method is used to execute `SELECT` statements, but we process the returned object with various `fetch` methods to see the results.

In [ ]:
# run queries by executing SQL and fetching results
SQL = ("SELECT title, score FROM movie")
res = cur.execute(SQL)

# fetchall gets all results as a list of tuples
res.fetchall()

In [ ]:
# add more data using a different method: executemany
data = [
    ("Monty Python Live at the Hollywood Bowl", 1982, 7.9),
    ("Monty Python's The Meaning of Life", 1983, 7.5),
    ("Monty Python's Life of Brain", 1979, 8.0)
]
# ? characters are replaced by values in data
cur.executemany("INSERT INTO movie VALUES(?, ?, ?)", data)

# always commit the change
con.commit()

Results are returned as a list of tuples that can be iterated through, e.g. using a `for` loop.

In [ ]:
res = cur.execute("SELECT year, title FROM movie ORDER BY year")
for row in res:
    print(row)

### Close the Connection

Always end the session by closing the connection.

In [ ]:
con.close()